In [14]:
%reload_ext autoreload
%autoreload 2

import os
import pprint
import shutil

import pymongo
import pandas as pd
from IPython import display

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad

from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap

## Run clirad-lw

In [95]:
def get_params():
    '''
    Define the clirad-lw case(s) to be run by
    specifying one or more input parameter dictionary.
    '''
    band = 6
    
    from climatools.atm.absorbers import nongreys_byband
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    commitnumber = '1013f91'
    molconc = nongreys_byband()[band]
    params = [{'commitnumber': commitnumber,
               'band': [band],
               'molecule': {mol: conc},
               'atmpro': atmpro,
               'tsfc': tsfc} 
              for mol, conc in molconc.items()
              for atmpro, tsfc in atmpros]
    
    if len(molconc) > 1:
        params.extend(
            [{'commitnumber': commitnumber, 
              'band': [band], 
              'molecule': molconc, 
              'atmpro': atmpro, 
              'tsfc': tsfc} for atmpro, tsfc in atmpros])
    return params

In [96]:
def script():
    for param in get_params():
        try:
            fdir = pipeline.get_fortran_dir(param=param, setup=setup)
            shutil.rmtree(fdir)
        except FileNotFoundError: pass
        
        fproc = pipeline.run_fortran(param=param, setup=setup)

In [97]:
script()

{'atmpro': 'mls',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [6],
 'commitnumber': '1013f91',
 'molecule': {'co2': 0.0004, 'h2o': 'atmpro'},
 'tsfc': 300}


## Save results in Mongodb

In [98]:
client = pymongo.MongoClient('localhost', 27017)
db = client.cliradnew
collection = db.lw

In [99]:
def make_doc(param=None):
    '''
    Make Mongodb document for a cliradlw run.
    
    Parameters
    ----------
    param: dict
        clirad-lw input parameters.
    doc: dict
        clirad-lw input (`param`) and output (flux and cooling rate)
    '''
    dir_fortran = pipeline.get_fortran_dir(param=param, 
                                           setup=setup)
    
    fnames = ['output_flux.dat', 'output_coolr.dat']
    
    s_outputs = []
    for fname in fnames:
        with open(os.path.join(dir_fortran, fname),
                  mode='r', encoding='utf-8') as f:
            s = f.read()
            
        s_outputs.append(s)
        
    doc = {fname.split('.')[0]: s
           for fname, s in zip(fnames, s_outputs)}
    doc['param'] = param
    return doc

def make_query(param=None):
    return {f'param.{n}': v for n, v in param.items()}

In [100]:
docs = (make_doc(param=param) for param in get_params())
collection.insert_many(docs)

In [101]:
collection.count()

243

In [41]:
#for param in params:
#    collection.delete_one(make_query(param=param))

## Run analysis notebooks